## <center>CSE 546: Reinforcement Learning</center>
### <center>Prof. Alina Vereshchaka</center>
<!-- ### <center>Fall 2022</center> -->

Welcome to the Assignment 2, Part 1: Introduction to Deep Reinforcement Learning and Neural Networks! The goal of this assignment is to make you comfortable with the application of different Neural Network structures depending on how the Reinforcement Learning environment is set up.

In [3]:
# Imports 
import cv2
import gym
from gym import spaces
from environment import WumpusWorldEnvironment 
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [4]:
!pip install environment
!pip install gym

# Environment

We will be working with an implementation of the Wumpus World environment. The environment comes from the book "Artificial Intelligence: A Modern Approach" by Stuart J. Russell and Peter Norvig. 

### ENVIRONMENT DETAILS:

The environment is a 6 x 6 grid world containing a total of 36 grid blocks. 

#### ENVIRONMENT OBJECTS:
The environment consists of the following objects:

1. **Agent** - The agent starts in the grid block at the bottom left corner whose co-ordinates are [0, 0]. The goal of our agent is to collect the Gold while avoiding the Wumpus and the pits. 

2. **Wumpus** - The monster which would eat the agent if they are in the same grid block.

3. **Pit** - The agent must avoid falling into the pits. 

4. **Gold** - The agent must collect the Gold.

5. **Breeze** - Breeze surrounds the Pits and warn the agent of a Pit in an adjacent grid block.

6. **Stench** - Stench surrounds the Wumpus and warns the agent of the Wumpus in an adjacent grid block.

#### ENVIRONMENT OBSERVATIONS:

Our implementation of the environment provides you with four different types of observations:

1. **Integer** - Integer in the range [0 - 35]. This represents the grid block the agent is in. E.g., if the agent is in the bottom left grid block (starting position) the observation would be 0, if the agent is in the grid block containing the Gold the observation would be 34, if the agent is in the top right grid block the observation would be 35.

2. **Vector** - 

    **2.1.** A vector of length 2 representing the agent co-ordinates. The first entry represents the x co-ordinate and the second entry represets the y co-ordinate. E.g., if the agent is in the bottom left grid block (starting position) the observation would be [0, 0], if the agent is in the grid block containing the Gold the observation would be [4, 5], if the agent is in the top right grid block the observation would be [5, 5].
    
    **2.2.** A vector of length 36 representing the one-hot encoding of the integer observation (refer type 1 above). E.g., if the agent is in the bottom left grid block (starting position) the observation would be [1, 0, ..., 0, 0], if the agent is in the grid block containing the Gold the observation would be [0, 0, ..., 1, 0], if the agent is in the top right grid block the observation would be [0, 0, ..., 0, 1].


3. **Image** - Image render of the environment returned as an NumPy array. The image size is 84 * 84 (same size used in the DQN paper). E.g., if the agent is in the bottom right grid block the observation is:

    Observation: (84 * 84)

     [[255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     ...

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]

     [255 255 255 ... 255 255 255]]

    Observation type: <class 'numpy.ndarray'>

    Observation Shape: (84, 84)

    Visually, it looks like:
    <img src="./images/environment_render.png" width="500" height="500">
    

4. **Float** - Float in the range [0 - $\infty$] representing the time elapsed in seconds. 

#### ENVIRONMENT ACTIONS:

Our implementation of the environment provides you with three different types of actions:

1. **Discrete** - Integer in the range [0 - 3] representing the four actions possible in the environment as follows: 0 - Right 1 - Left 2 - Up 3 - Down.

2. **Multi-Discrete** - Array of length four where each element takes binary values 0 or 1. Array elements represent if we take a particular action. Array element with index 0 corresponds to the right action, index 1 corresponds to the left action, index 2 corresponds to the up action, and index 3 corresponds to the down action. E.g., 
   action = [1, 0, 0, 0] would result in the agent moving right.
   action = [1, 0, 1, 0] would result in the agent moving right and up.
   action = [0, 1, 0, 1] would result in the agent moving left and down.

3. **Continuous** - Float in the range [-1, 1] determining whether the agent will go left, right, up, or down as follows:

    if -1 <= action <= -0.5:
        Go Right.
    elif -0.5 < action <= 0:
        Go Left.
    elif 0 < action <= 0.5:
        Go Up.
    elif 0.5 < action <= 1:
        Go Down.
        
### YOUR TASK IS TO USE A NEURAL NETWORK TO WORK WITH ALL FOUR TYPES OF OBSERVATIONS AND ALL THREE TYPES OF  ACTIONS.

<img src="./images/wumpus_world_environment.jpg" width="600" height="600">

# START COMPLETING YOUR ASSIGNMENT HERE

## Observation Type - Integer, Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is an integer (refer to environment observations type 1). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_1_64_4.png">

In [12]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='integer', action_type='discrete')
obs = environment.reset()
print(obs)

# BEGIN_YOUR_CODE
class NeuralNetwork1(nn.Module):
    def __init__(self) -> None:
        super(NeuralNetwork1 , self).__init__()
        self.fc1 = nn.Linear(in_features= 1, out_features= 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features= 64 , out_features= 4)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

environment = WumpusWorldEnvironment(observation_type='integer', action_type='discrete')
observation = environment.reset()
random_action = np.random.randint(3)
observation , reward, done , info = environment.step(random_action)
model = NeuralNetwork1()
output = model(torch.Tensor([observation]))
print(output)
# END_YOUR_CODE

0
tensor([0.1511, 0.0388, 0.1365, 0.0206], grad_fn=<AddBackward0>)


## Observation Type - Vector (2.1), Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 2 (refer to environment observations type 2.1). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_2_64_4.png">

In [6]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='vector', action_type='discrete')
observation = environment.reset()
# BEGIN_YOUR_CODE
class NeuralNetwork2(nn.Module):
    def __init__(self) -> None:
        super(NeuralNetwork2 , self).__init__()
        self.fc1 = nn.Linear(in_features= 2, out_features= 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features= 64 , out_features= 4)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

environment = WumpusWorldEnvironment(observation_type='vector', action_type='discrete')
observation = environment.reset()
random_action = np.random.randint(3)
observation , reward, done , info = environment.step(random_action)
model = NeuralNetwork2()
output = model(torch.Tensor([observation]))
print(output)
# END_YOUR_CODE

tensor([[ 0.2368,  0.1204,  0.0432, -0.3857]], grad_fn=<AddmmBackward0>)


C:\Users\chinm\AppData\Local\Temp\ipykernel_14592\2554662940.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  output = model(torch.Tensor([observation]))


## Observation Type - Vector (2.2), Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector.

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_4.png">

In [9]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

environment = WumpusWorldEnvironment(observation_type='integer', action_type='discrete')
observation = environment.reset()

# BEGIN_YOUR_CODE
class NeuralNetwork3(nn.Module):
    def __init__(self) -> None:
        super(NeuralNetwork3 , self).__init__()
        self.fc1 = nn.Linear(in_features= 36, out_features= 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features= 64 , out_features= 4)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

environment = WumpusWorldEnvironment(observation_type='integer', action_type='discrete')
observation = environment.reset()
random_action = np.random.randint(3)
observation = torch.Tensor([observation]).to(torch.int64)
observation = nn.functional.one_hot(observation,36).to(torch.float)
model = NeuralNetwork3()
output = model(observation)
print(output)
# END_YOUR_CODE

tensor([[0.5273, 0.4619, 0.5220, 0.5419]], grad_fn=<SigmoidBackward0>)


## Observation Type - Image, Action Type - Discrete

The part of the assignment requires you to create a convolutional neural network with one convolutional layer having 128 filters of size 3 x 3, one hidden layer having 64 neurons, and the output layer having 4 neurons. The input to the neural network is an image of size 84 * 84 (refer to environment observations type 3). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/convolutional_neural_network_84x84_128_64_4.png">

In [11]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""
# BEGIN_YOUR_CODE
class ConvolutionNeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super(ConvolutionNeuralNetwork , self).__init__()
        self.conv1 = nn.Conv2d(1,128,kernel_size=3)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = x.flatten()
        self.nn1 = nn.Linear(in_features= x.shape[0] , out_features= 64)
        x = self.nn1(x)
        self.nn2 = nn.Linear(in_features=64 , out_features=4)
        x = self.nn2(x)
        return x

environment = WumpusWorldEnvironment(observation_type='image', action_type='discrete')
observation = environment.reset()
model = ConvolutionNeuralNetwork()
random_action = np.random.randint(3)
observation ,reward , done , info  = environment.step(random_action)
observation = torch.from_numpy(observation)
observation = observation / 255
observation = observation.unsqueeze(0)

output = model(observation)
print(output)
# END_YOUR_CODE

tensor([ 0.1151, -0.0942, -0.0446, -0.0693], grad_fn=<AddBackward0>)


## Observation Type - Float, Action Type - Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 256 neurons and the output layer having 4 neurons. The input to the neural network is a float (refer to environment observations type 4). The output of the neural network is an array represeting the Q-values from which you will choose an action (refer to environment actions type 1).

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_1_64_4.png">

In [10]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted Q-values for the four actions. Print the observation and the Q-values."""

# BEGIN_YOUR_CODE
class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super(NeuralNetwork , self).__init__()
        self.fc1 = nn.Linear(in_features= 1, out_features= 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features= 256 , out_features= 4)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

environment = WumpusWorldEnvironment(observation_type='float', action_type='discrete')
observation = environment.reset()
random_action = np.random.randint(3)
observation , reward, done , info = environment.step(random_action)
model = NeuralNetwork()
output = model(torch.Tensor([observation]))
print(output)

# END_YOUR_CODE

tensor([ 0.1359, -0.1016, -0.0160,  0.2514], grad_fn=<AddBackward0>)


## Observation Type - Vector (2.2), Action Type - Multi-Discrete

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 4 neurons. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an array representing the probability of choosing the actions. (If the value of the array element is >=0.5 you will perform the action.) (refer to environment actions type 2).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector.

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_4_sigmoid.png">

In [ ]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted action probabilities for the four actions. Print the observation and the action probabilities."""


# BEGIN_YOUR_CODE

class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super(NeuralNetwork , self).__init__()
        self.fc1 = nn.Linear(in_features= 36, out_features= 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features= 64 , out_features= 4)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x
environment = WumpusWorldEnvironment(observation_type='integer', action_type='multi_discrete')
observation = environment.reset()
observation = torch.Tensor([observation]).to(torch.int64)
observation = nn.functional.one_hot(observation,36).to(torch.float)
model = NeuralNetwork()
output = model(observation)
print(output)
# END_YOUR_CODE

tensor([[0.4719, 0.4788, 0.4764, 0.5164]], grad_fn=<SigmoidBackward0>)


## Observation Type - Vector (2.2), Action Type - Continuous

The part of the assignment requires you to create a sequential dense neural network with 1 hidden layer having 64 neurons and the output layer having 1 neuron. The input to the neural network is a vector of length 36 (refer to environment observations type 2.2). The output of the neural network is an float in the range [-1, 1] determining the action which will be taken. (refer to environment actions type 3).

**HINT:** Use the integer observation and convert it to a one-hot encoded vector and use the TanH activation function to get the output in the range [-1, 1].

The following figure shows the network structure you will have to use:

<img src="./images/neural_network_structures/neural_network_36_64_1.png">

In [ ]:
"""TO DO: Create a neural network, pass it the observation from the environment
and get the predicted action. Print the observation and the predicted action."""

# BEGIN_YOUR_CODE
class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super(NeuralNetwork , self).__init__()
        self.fc1 = nn.Linear(in_features= 36, out_features= 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features= 64 , out_features= 1)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        return x
environment = WumpusWorldEnvironment(observation_type='integer', action_type='multi_discrete')
observation = environment.reset()
observation = torch.Tensor([observation]).to(torch.int64)
observation = nn.functional.one_hot(observation,36).to(torch.float)
model = NeuralNetwork()
output = model(observation)
print(output)

# END_YOUR_CODE

tensor([[-0.0326]], grad_fn=<AddmmBackward0>)
